In [2]:
from qdrant_client import QdrantClient, models

qd_client = QdrantClient(url="http://localhost:6333")

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [48]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)


In [50]:
EMB_DIM = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"
collection_name = "zoomcamp-rag-final-2"

qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size = EMB_DIM,
        distance = models.Distance.COSINE # -1 a 1
    )
)

True

In [51]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [52]:
qd_client.upsert(collection_name=collection_name, points=points)

: 

In [21]:
question = "I just discovered the course. Can I Still join ? "

results = qd_client.query_points(
        collection_name = collection_name,
        query = models.Document(
            text = question,
            model = model_handle
        ),
        limit = 5,
        with_payload = True
    )

In [22]:
results

QueryResponse(points=[ScoredPoint(id=450, version=0, score=0.8426586, payload={'text': 'The course is available in the self-paced mode too, so you can go through the materials at any time. But if you want to do it as a cohort with other students, the next iterations will happen in September 2023, September 2024 (and potentially other Septembers as well).', 'course': 'machine-learning-zoomcamp', 'section': 'General course-related questions'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=7, version=0, score=0.83405924, payload={'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.', 'course': 'data-engineering-zoomcamp', 'section': 'General course-related questions'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=45

In [23]:
results_list = []

for point in results.points:
    results_list.append(point.payload)

In [24]:
results_list

[{'text': 'The course is available in the self-paced mode too, so you can go through the materials at any time. But if you want to do it as a cohort with other students, the next iterations will happen in September 2023, September 2024 (and potentially other Septembers as well).',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions'},
 {'text': 'We won’t re-record the course videos. The focus of the course and the skills we want to teach remained the same, and the videos are still up-to-date.\nIf you haven’t taken part in the previous iteration, you can start watchin

In [25]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [45]:
question = "I just discovered the course. Can I Still join? "


def vector_search(query, limit = 1):
    course = "mlops-zoomcamp"
    resultados = qd_client.query_points(
        collection_name = collection_name,
        query = models.Document(
            text = query,
            model = model_handle
        ),
        query_filter = models.Filter(
            must = [
                models.FieldCondition(
                    key = "course",
                    match = models.MatchValue(value = course))
                    ]
        ),
        limit = 5,
        with_payload = True
    )
    results = []
    print("resultados.points:", resultados.points)
    for point in resultados.points:
        results.append(point.payload)

    return results

In [27]:
vector_search('How do I Run Kafka', course = "data-engineering-zoomcamp")

resultados.points: [ScoredPoint(id=857, version=0, score=0.77556765, payload={'text': "You have something running on the 5000 port. You need to stop it.\nAnswer: On terminal in mac .\nRun ps -A | grep gunicorn\nLook for the number process id which is the 1st number after running the command\nkill 13580\nwhere 13580  represents the process number.\nSource\nwarrie.warrieus@gmail.com\nOr by executing the following command it will kill all the processes using port 5000:\n>> sudo fuser -k 5000/tcp\nAnswered by Vaibhav Khandelwal\nJust execute in the command below in he command line to kill the running port\n->> kill -9 $(ps -A | grep python | awk '{print $1}')\nAnswered by kamaldeen (kamaldeen32@gmail.com)\nChange to different port (5001 in this case)\n>> mlflow ui --backend-store-uri sqlite:///mlflow.db --port 5001\nAnswered by krishna (nellaikrishna@gmail.com)", 'course': 'mlops-zoomcamp', 'section': 'Module 2: Experiment tracking'}, vector=None, shard_key=None, order_value=None), ScoredP

[{'text': "You have something running on the 5000 port. You need to stop it.\nAnswer: On terminal in mac .\nRun ps -A | grep gunicorn\nLook for the number process id which is the 1st number after running the command\nkill 13580\nwhere 13580  represents the process number.\nSource\nwarrie.warrieus@gmail.com\nOr by executing the following command it will kill all the processes using port 5000:\n>> sudo fuser -k 5000/tcp\nAnswered by Vaibhav Khandelwal\nJust execute in the command below in he command line to kill the running port\n->> kill -9 $(ps -A | grep python | awk '{print $1}')\nAnswered by kamaldeen (kamaldeen32@gmail.com)\nChange to different port (5001 in this case)\n>> mlflow ui --backend-store-uri sqlite:///mlflow.db --port 5001\nAnswered by krishna (nellaikrishna@gmail.com)",
  'course': 'mlops-zoomcamp',
  'section': 'Module 2: Experiment tracking'},
 {'text': 'Ensure the correct image is being used to derive from.\nCopy the data from local to the docker image using the COPY 

In [28]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [38]:
import os
print("Minha chave é:", os.getenv("OPENAI_API_KEY"))

Minha chave é: None


In [42]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)  # carrega o .env
from openai import OpenAI

openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def llm(prompt):
    response = openai_client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [46]:
def rag (query):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [47]:
rag('how do I run kafka?')

resultados.points: [ScoredPoint(id=857, version=0, score=0.7742764, payload={'text': "You have something running on the 5000 port. You need to stop it.\nAnswer: On terminal in mac .\nRun ps -A | grep gunicorn\nLook for the number process id which is the 1st number after running the command\nkill 13580\nwhere 13580  represents the process number.\nSource\nwarrie.warrieus@gmail.com\nOr by executing the following command it will kill all the processes using port 5000:\n>> sudo fuser -k 5000/tcp\nAnswered by Vaibhav Khandelwal\nJust execute in the command below in he command line to kill the running port\n->> kill -9 $(ps -A | grep python | awk '{print $1}')\nAnswered by kamaldeen (kamaldeen32@gmail.com)\nChange to different port (5001 in this case)\n>> mlflow ui --backend-store-uri sqlite:///mlflow.db --port 5001\nAnswered by krishna (nellaikrishna@gmail.com)", 'course': 'mlops-zoomcamp', 'section': 'Module 2: Experiment tracking'}, vector=None, shard_key=None, order_value=None), ScoredPo

KeyError: 'question'